In [ ]:
# berlin speech database
import os
from pydub import AudioSegment

su = 0
for j, fn in enumerate(os.listdir('wav')):
  audio = AudioSegment.from_wav('wav/'+fn)
  if len(audio)>5000:
    trimmed_audio = audio[:5000]
    trimmed_audio.export("AllSpeechFiles/berlin"+fn,format = "wav")
  else:
    audio.export("AllSpeechFiles/berlin"+fn,format = "wav")
  su = su+1

# ravdess speech database
for i in range(1,25):
  if i<10:
    directory = "Actor_0"+str(i)
  else:
    directory = "Actor_"+str(i)
  for j, fn in enumerate(os.listdir(directory)):
    audio = AudioSegment.from_wav(directory+'/'+fn)
    if len(audio)>5000:
      trimmed_audio = audio[:5000]
      trimmed_audio.export("AllSpeechFiles/ravdess"+fn,format = "wav")
    else:
      audio.export("AllSpeechFiles/ravdess"+fn,format = "wav")
    su = su+1

# TESS toronto database
for subdir, dirs, files in os.walk("TESS Toronto emotional speech set data"):
  for file in files:
    if os.path.join(subdir, file) == "TESS Toronto emotional speech set data/YAF_fear/YAF_neat_fear.wav":
      continue
    if os.path.join(subdir, file) == "TESS Toronto emotional speech set data/YAF_angry/YAF_germ_angry.wav":
      continue
    audio = AudioSegment.from_wav(os.path.join(subdir, file))
    if len(audio)>5000:
      trimmed_audio = audio[:5000]
      trimmed_audio.export("AllSpeechFiles/TESS"+file,format = "wav")
    else:
      audio.export("AllSpeechFiles/TESS"+file,format = "wav")
    su = su+1

# cremad database
for j, fn in enumerate(os.listdir('AudioWAV')):
  audio = AudioSegment.from_wav('AudioWAV/'+fn)
  if len(audio)>5000:
    trimmed_audio = audio[:5000]
    trimmed_audio.export("AllSpeechFiles/cremad"+fn,format = "wav")
  else:
    audio.export("AllSpeechFiles/cremad"+fn,format = "wav")
  su = su+1

# Savee database
for subdir, dirs, files in os.walk("AudioData"):
  for dir in dirs:
    for j, fn in enumerate(os.listdir('AudioData/'+dir)):
      audio = AudioSegment.from_wav('AudioData/'+dir+'/'+fn)
      if len(audio)>5000:
        trimmed_audio = audio[:5000]
        trimmed_audio.export("AllSpeechFiles/savee"+dir+fn,format = "wav")
      else:
        audio.export("AllSpeechFiles/savee"+dir+fn,format = "wav")
      su = su+1

In [ ]:
import numpy
import scipy.io.wavfile
from scipy.fftpack import dct
import os
import matplotlib.pyplot as plt
import librosa
import librosa.display

for j, fn in enumerate(os.listdir('AllSpeechFiles')):
  audio = AudioSegment.from_wav('AllSpeechFiles/'+fn)
  sample_rate, signal = scipy.io.wavfile.read('AllSpeechFiles/'+fn)
  pre_emphasis = 0.97
  frame_size = 0.025
  frame_stride = 0.01
  NFFT = 512
  nfilt = 40
  if len(audio) == 5000:
    sample_rate = 16000
  else:
    a = len(audio)//1000
    sample_rate = 80000//a
  emphasized_signal = numpy.append(signal[0], signal[1:] - pre_emphasis * signal[:-1])
  frame_length, frame_step = frame_size * sample_rate, frame_stride * sample_rate  # Convert from seconds to samples
  signal_length = len(emphasized_signal)
  frame_length = int(round(frame_length))
  frame_step = int(round(frame_step))
  num_frames = int(numpy.ceil(float(numpy.abs(signal_length - frame_length)) / frame_step))  # Make sure that we have at least 1 frame

  pad_signal_length = num_frames * frame_step + frame_length
  z = numpy.zeros((pad_signal_length - signal_length))
  pad_signal = numpy.append(emphasized_signal, z) # Pad Signal to make sure that all frames have equal number of samples without truncating any samples from the original signal

  indices = numpy.tile(numpy.arange(0, frame_length), (num_frames, 1)) + numpy.tile(numpy.arange(0, num_frames * frame_step, frame_step), (frame_length, 1)).T
  frames = pad_signal[indices.astype(numpy.int32, copy=False)]
  frames *= numpy.hamming(frame_length)
  mag_frames = numpy.absolute(numpy.fft.rfft(frames, NFFT))  # Magnitude of the FFT
  pow_frames = ((1.0 / NFFT) * ((mag_frames) ** 2))  # Power Spectrum
  low_freq_mel = 0
  high_freq_mel = (2595 * numpy.log10(1 + (sample_rate / 2) / 700))  # Convert Hz to Mel
  mel_points = numpy.linspace(low_freq_mel, high_freq_mel, nfilt + 2)  # Equally spaced in Mel scale
  hz_points = (700 * (10**(mel_points / 2595) - 1))  # Convert Mel to Hz
  bin = numpy.floor((NFFT + 1) * hz_points / sample_rate)

  fbank = numpy.zeros((nfilt, int(numpy.floor(NFFT / 2 + 1))))
  for m in range(1, nfilt + 1):
      f_m_minus = int(bin[m - 1])   # left
      f_m = int(bin[m])             # center
      f_m_plus = int(bin[m + 1])    # right

      for k in range(f_m_minus, f_m):
          fbank[m - 1, k] = (k - bin[m - 1]) / (bin[m] - bin[m - 1])
      for k in range(f_m, f_m_plus):
          fbank[m - 1, k] = (bin[m + 1] - k) / (bin[m + 1] - bin[m])
  filter_banks = numpy.dot(pow_frames, fbank.T)
  filter_banks = numpy.where(filter_banks == 0, numpy.finfo(float).eps, filter_banks)  # Numerical Stability
  filter_banks = 20 * numpy.log10(filter_banks)  # dB
  #filter_banks -= (numpy.mean(filter_banks, axis=0) + 1e-8)
  sgram_mag, _ = librosa.magphase(filter_banks)
  mel_scale_sgram = librosa.feature.melspectrogram(S=sgram_mag, sr=sample_rate)
  librosa.display.specshow(mel_scale_sgram)
  plt.colorbar(format='%+2.0f dB')
  break

In [ ]:
# emo-db
categories = {
    "W": 5,
    "L": 4,
    "E": 7,
    "A": 6,
    "F": 3,
    "T": 4,
    "N": 1
}

# ravdess
emotions={
  '01':1,
  '03':3,
  '04':4,
  '05':5,
  '06':6,
  '07':7,
  '08':8
}

for file in glob.glob("Actor_*/*.wav"):
  file_name=os.path.basename(file)
  if file_name.split("-")[2] == '02':
    continue
  emotion=emotions[file_name.split("-")[2]]
  x.append(file)
  y.append(emotion)

#cremad
emotions={
  'NEU':1,
  'HAP':3,
  'SAD':4,
  'ANG':5,
  'FEA':6,
  'DIS':7,
}

for file in glob.glob("AudioWAV/*.wav"):
  file_name=os.path.basename(file)
  emotion=emotions[file_name.split("_")[2]]
  x.append(file)
  y.append(emotion)

#savee
emotions={
  'n':1,
  'h':3,
  'sa':4,
  'a':5,
  'f':6,
  'd':7,
  'su':8
}
l = ['n','h','a','f','d']
for file in glob.glob("AudioData/*/*.wav"):
  file_name=os.path.basename(file)
  if file_name[0] in l:
    emotion=emotions[file_name[0]]
  else:
    emotion = emotions[file_name[0]+file_name[1]]
  x.append(file)
  y.append(emotion)

# tess
emotions={
  'neutral':1,
  'happy':3,
  'sad':4,
  'angry':5,
  'fear':6,
  'disgust':7,
  'ps':8
}

for file in glob.glob("TESS Toronto emotional speech set data/*/*.wav"):
  file_name=os.path.basename(file)
  emotion=emotions[file_name.split("_")[2][:-4]]
  x.append(file)
  y.append(emotion)

In [ ]:
from fastai.vision.all import *
from fastaudio.core.all import *
from fastaudio.augment.all import *

class StatsRecorder:
    def __init__(self, red_dims=(0,2,3)):
        """Accumulates normalization statistics across mini-batches.
        ref: http://notmatthancock.github.io/2017/03/23/simple-batch-stat-updates.html
        """
        self.red_dims = red_dims # which mini-batch dimensions to average over
        self.nobservations = 0   # running number of observations

    def update(self, data):
        """
        data: ndarray, shape (nobservations, ndimensions)
        """
        # initialize stats and dimensions on first batch
        if self.nobservations == 0:
            self.mean = data.mean(dim=self.red_dims, keepdim=True)
            self.std  = data.std (dim=self.red_dims,keepdim=True)
            self.nobservations = data.shape[0]
            self.ndimensions   = data.shape[1]
        else:
            if data.shape[1] != self.ndimensions:
                raise ValueError('Data dims do not match previous observations.')
            
            # find mean of new mini batch
            newmean = data.mean(dim=self.red_dims, keepdim=True)
            newstd  = data.std(dim=self.red_dims, keepdim=True)
            
            # update number of observations
            m = self.nobservations * 1.0
            n = data.shape[0]

            # update running statistics
            tmp = self.mean
            self.mean = m/(m+n)*tmp + n/(m+n)*newmean
            self.std  = m/(m+n)*self.std**2 + n/(m+n)*newstd**2 +\
                        m*n/(m+n)**2 * (tmp - newmean)**2
            self.std  = torch.sqrt(self.std)
                                 
            # update total number of seen samples
            self.nobservations += n

class AudioNormalize(Transform):
    "Normalizes a single `AudioTensor`."
    def encodes(self, x:AudioTensor): return (x-x.mean()) / x.std()

def CrossValidationSplitter(col='fold', fold=1):
    "Split `items` (supposed to be a dataframe) by fold in `col`"
    def _inner(o):
        assert isinstance(o, pd.DataFrame), "ColSplitter only works when your items are a pandas DataFrame"
        col_values = o.iloc[:,col] if isinstance(col, int) else o[col]
        valid_idx = (col_values == fold).values.astype('bool')
        return IndexSplitter(mask2idxs(valid_idx))(o)
    return _inner

cfg = AudioConfig.BasicSpectrogram() # with default torchaudio parameters
audio2spec = AudioToSpec.from_cfg(cfg)
path = "AllSpeechFiles/"
auds = DataBlock(blocks=(AudioBlock, CategoryBlock),  
                 get_x=ColReader("file_name"), 
                 item_tfms = [AudioNormalize],
                 batch_tfms = [audio2spec],
                 get_y=ColReader("emotion"))
dbunch = auds.dataloaders(df, bs=64)

In [ ]:
# create recorders
global_stats  = StatsRecorder()
channel_stats = StatsRecorder(red_dims=(0,1,3))

# step through the training dataset
with torch.no_grad():
    for idx,(x,y) in enumerate(iter(dbunch.train)):
        # update normalization statistics
        global_stats.update(x)
        channel_stats.update(x)
    
# parse out both sets of stats
global_mean,global_std = global_stats.mean,global_stats.std
channel_mean,channel_std = channel_stats.mean,channel_stats.std

In [ ]:
import pydub 
import numpy as np
from numpy import save
import audio2numpy as a2n

def read(f, normalized=False):
    """MP3 to numpy array"""
    a = pydub.AudioSegment.from_mp3(f)
    y = np.array(a.get_array_of_samples())
    if a.channels == 2:
        y = y.reshape((-1, 2))
    if normalized:
        return a.frame_rate, np.float32(y) / 2**15
    else:
        return a.frame_rate, y

for i in range(len(x)):
  npym,sr=a2n.audio_from_file(x[i])
  name = x[i].split('/')[-1]
  name = name[:-4]
  save("npydirec/"+name+'.npy', npym)

In [ ]:
# ravdess
emotions={
  '01':'neutral',
  '03':'Happy',
  '04':'Sad',
  '05':'Angry',
  '06':'Fear',
  '07':'Disgust',
  '08':'Surprised'
}

for file in glob.glob("Actor_*/*.wav"):
  file_name=os.path.basename(file)
  if file_name.split("-")[2] == '02':
    continue
  emotion=emotions[file_name.split("-")[2]]
  if emotion == 'Surprised':
    continue
  x.append(file)
  y.append(emotion)
  #shutil.copy(file,'AllSpeechFiles/')

#savee
emotions={
  'n':'Neutral',
  'h':'Happy',
  'sa':'Sad',
  'a':'Angry',
  'f':'Fear',
  'd':'Disgust',
  'su':'Surprised'
}

l = ['n','h','a','f','d']
for file in glob.glob("AudioData/*/*.wav"):
  file_name=os.path.basename(file)
  if file_name[2] in l:
    emotion=emotions[file_name[2]]
  else:
    emotion = emotions[file_name[2]+file_name[3]]
  if emotion=='Surprised':
    continue
  x.append(file)
  y.append(emotion)
  #shutil.copy(file,'AllSpeechFiles/')

# tess
emotions={
  'neutral':'Neutral',
  'happy':'Happy',
  'sad':'Sad',
  'angry':'Angry',
  'fear':'Fear',
  'disgust':'Disgust',
  'ps':'Surprised'
}

for file in glob.glob("TESS Toronto emotional speech set data/*/*.wav"):
  file_name=os.path.basename(file)
  emotion=emotions[file_name.split("_")[2][:-4]]
  if emotion=='Surprised':
    continue
  x.append(file)
  y.append(emotion)

# ravdess
emotions={
  '01':'neutral',
  '03':'Happy',
  '04':'Sad',
  '05':'Angry',
  '06':'Fear',
  '07':'Disgust',
  '08':'Surprised'
}

for file in glob.glob("Actor_*/*.wav"):
  file_name=os.path.basename(file)
  if file_name.split("-")[2] == '02':
    continue
  emotion=emotions[file_name.split("-")[2]]
  if emotion == 'Surprised':
    continue
  x.append(file)
  y.append(emotion)
  #shutil.copy(file,'AllSpeechFiles/')